In [1]:
import sqlite3

In [2]:
START_YEAR = 2005
CURRENT_SEASON = 2024
COMPLETE_CURRENT_SEASON = False
SPORT_IDS = [1,11,12,13,14,15,16]
MONTHS = [4,5,6,7,8,9] # March folded into April, October into September

In [3]:
db = sqlite3.connect("BaseballStats.db")
cursor = db.cursor()

In [4]:
for year in range(START_YEAR, CURRENT_SEASON + 1):
    for level in SPORT_IDS:
        for month in MONTHS:
            # Check if already exists
            if cursor.execute("SELECT COUNT(*) FROM Level_PitcherStats WHERE Year=? AND Month=? AND Level=?", (year, month, level)).fetchone()[0] > 0:
                continue
            
            bf, outs, go, ao, r, er, h, k, bb, hbp, doubles, triples, hr = cursor.execute('SELECT SUM(battersFaced), SUM(outs), SUM(go), SUM(ao), SUM(r), SUM(er), SUM(h), SUM(k), SUM(bb), SUM(hbp), SUM("2B"), SUM("3B"), SUM(HR) FROM Player_Pitcher_MonthStats WHERE Year=? AND Month=? AND Level=?', (year, month, level)).fetchone()
            
            # Some year/level combos do not have any games
            if bf == 0 or bf == None:
                continue
            
            # Calculate stats
            ab = bf - bb - hbp
            singles = h - doubles - triples - hr
            iso = (doubles + 2 * triples + 3 * hr) / ab
            avg = h / ab
            obp = (h + bb + hbp) / bf
            slg = (singles + 2 * doubles + 3 * triples + 4 * hr) / ab
            hrPerc = hr / bf
            bbPerc = bb / bf
            kPerc = k / bf
            goPerc = go / (go + ao)
            # https://library.fangraphs.com/offense/woba/
            wOBA = (0.69 * bb + 0.72 * hbp + 0.89 * singles + 1.27 * doubles + 1.62 * triples + 2.10 * hr) / (bf)
            era = er / (outs / 27)
            ra = r / (outs / 27)
            fipNoConstant = (13 * hr + 3 * (bb + hbp) - 2 * k) / (outs / 3)
            fipConstant = era - fipNoConstant
            
            # Insert data
            cursor.execute("INSERT INTO Level_PitcherStats VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?)", (level, year, month, era, ra, fipConstant, wOBA, hrPerc, bbPerc, kPerc, goPerc, avg, iso))
            db.commit()
            cursor = db.cursor()